### 단어\_태그\_T/F(받침여부)_원단어의발음
### https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit#gid=4
# 명사 NNG, 동사 VV, 형용사 VA


In [1]:
from konlpy.tag import Mecab
import pickle
import re
import sys
from gensim.models import TfidfModel
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
from pprint import pprint
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import mglearn
from pprint import pprint
import numpy as np
import gc
import copy
import pandas as pd


class SB_Word2Vec():    
    
    def __init__(self, morph_list):
        self.dct = Dictionary(morph_list)
        self.corpus = [self.dct.doc2bow(line) for line in morph_list]
        self.build_Word2Vec(morph_list)
    
    def make_Word2Vec(self, morph_list, size=50, window=2, min_count=10, iteration=100):
        self.em = Word2Vec(morph_list, size=size, window=window, min_count=min_count, iter=iteration)
        self.em_vocab = list(self.em.wv.vocab.keys())
        self.em_vocab_dic = {word:idx for idx, word in enumerate(self.em_vocab)}

    def make_Word2Sen_matrix(self): 
        vocab_size = len(self.em_vocab)
        self.sen_matrix = np.zeros((len(self.corpus), vocab_size))
        for idx, row in enumerate(self.sen_matrix):
            for idx2, frequency in self.corpus[idx]:
                    if self.dct[idx2] in self.em_vocab:
                        self.sen_matrix[idx][self.em_vocab_dic[self.dct[idx2]]] = frequency                
        self.sim_matrix = np.zeros((vocab_size, vocab_size))
        for idx, w1 in enumerate(self.em_vocab):
            for idx2, w2 in enumerate(self.em_vocab):
                self.sim_matrix[idx][idx2] =  self.em.wv.similarity(w1, w2)

        self.word2sen_matrix = np.dot(self.sim_matrix, np.transpose(self.sen_matrix))

        return self.word2sen_matrix

    def get_sim_sen(self, keyword, main_text, number=1):
        self.sim_sen_index = np.argsort(self.word2sen_matrix[self.em_vocab_dic[keyword]])
        self.most_sim_sen_index = np.argmax(self.word2sen_matrix[self.em_vocab_dic[keyword]])
        index_list = self.sim_sen_index.reshape((-1,)).tolist()
        index_list.reverse()
        
        for idx, i in enumerate(index_list[:number]):
            print(str(idx + 1))
            print(main_text[i])
        return index_list
    
    def build_Word2Vec(self, morph_list):
        self.make_Word2Vec(morph_list)
        self.make_Word2Sen_matrix()
        
        
class SB_LDA():

    def make_lda(self, morph_joined, ntopic=10, learning_method='batch', max_iter=25, random_state=0, n_words=20):        
        self.vect = CountVectorizer(max_features=10000, max_df=.15)
        self.X = self.vect.fit_transform(morph_joined)
        self.lda = LatentDirichletAllocation(n_components=ntopic, learning_method=learning_method, max_iter=max_iter, random_state=random_state)
        self.document_topics = self.lda.fit_transform(self.X)
        self.sorting = np.argsort(self.lda.components_, axis=1)[:, ::-1]
        self.feature_names = np.array(self.vect.get_feature_names())
        mglearn.tools.print_topics(topics=range(ntopic), feature_names=self.feature_names, sorting=self.sorting, topics_per_chunk=5, n_words=n_words)

    def related_doc(self, main_text_list, topic_index, number=10):
        category = np.argsort(self.document_topics[:, topic_index])[::-1]
        related_docs = []
        for i in category[:number]:
            print(i)
            print(main_text_list[i] + ".\n")
            related_docs.append((i, main_text_list[i]))
        return related_docs

class SB_Tfidf():    
    
    def __init__(self, list_morph_merged):
        self.list_morph_merged = list_morph_merged
        self.dct = Dictionary(self.list_morph_merged)
        self.corpus = [self.dct.doc2bow(line) for line in self.list_morph_merged]

    def get_tfidf(self):       
        self.model = TfidfModel(self.corpus)
        self.tfidf = []
        for i in self.corpus:
             self.tfidf.append(sorted(self.model[i], key = lambda x: x[1], reverse=True))
        self.tfidf_hangul = []
        for idx1, i in enumerate(self.tfidf):
            self.tfidf_hangul.append([(self.dct[j[0]], j[1]) for j in i])        
        
        return self.tfidf_hangul
    
def frequency(merged):
    word_count = Counter(merged)
    word_count2 = []
    for i in word_count:
        word_count2.append((i, word_count[i]))
    word_count2 = sorted(word_count2, key=lambda x: x[1], reverse = True)
    return word_count2




class Social_analysis():
    
    non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
    syn_dic = {}
    del_list = []
    ngram_dic = {}
    
    def __init__(self):
        self.mecab = Mecab()
        try:
            self.load_dictionary()
        except:
            print('dictionary error')
    def load_dictionary(self, path='Data/custom_dic.csv'):
        dic_df = pd.read_csv(path, encoding='cp949')
        for i in range(len(dic_df)):
            self.ngram_dic[dic_df.loc[i,'key']] =  dic_df.loc[i, 'value']
    
    def insta_DB_to_table(self, DBname='intake', keyword='intake'):
        import pymssql
        import pandas.io.sql as pdsql
        import pandas as pd
        self.query = \
        """
        SELECT user_id, created_at, main_text, hashtags, comments, likes, current_url FROM instaPosting WHERE keyword = '{}'
        """.format(keyword)
        conn = pymssql.connect("intakedb.c63elkxbiwfc.us-east-2.rds.amazonaws.com:1433", "gh", "ghintake", DBname)
        self.df = pdsql.read_sql_query(self.query, con=conn)
        # df['main_text'] = df.main_text.apply(lambda x: x.replace('#',' ').translate(self.non_bmp_map))
        # df['created_at'] = df.created_at.apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
        conn.close()
        self.raw_data = self.df.as_matrix()
    
    def shop_DB_to_table(self, DBname='intake', keyword='intake'):
        import pymssql
        import pandas.io.sql as pdsql
        import pandas as pd
        self.query = \
        """
        SELECT mall, corpName, productCode, date, id, productScore, recommScore, main_text FROM VproductReview WHERE corpName = '{}'
        """.format(keyword)
        conn = pymssql.connect("intakedb.c63elkxbiwfc.us-east-2.rds.amazonaws.com:1433", "gh", "ghintake", DBname)
        self.df = pdsql.read_sql_query(self.query, con=conn)
        # df['main_text'] = df.main_text.apply(lambda x: x.replace('#',' ').translate(self.non_bmp_map))
        # df['created_at'] = df.created_at.apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
        conn.close()
        self.raw_data = self.df.as_matrix()       
        
    def blog_DB_to_table(self, DBname='intake', keyword='intake'):
        import pymssql
        import pandas.io.sql as pdsql
        import pandas as pd
        self.query = \
        """
        SELECT keyword, created_at, post_name, main_text, current_url FROM NaverBlogReview WHERE keyword = '{}'
        """.format(keyword)
        conn = pymssql.connect("intakedb.c63elkxbiwfc.us-east-2.rds.amazonaws.com:1433", "gh", "ghintake", DBname)
        self.df = pdsql.read_sql_query(self.query, con=conn)
        # df['main_text'] = df.main_text.apply(lambda x: x.replace('#',' ').translate(self.non_bmp_map))
        # df['created_at'] = df.created_at.apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
        conn.close()
        self.raw_data = self.df.as_matrix()       
    
    def pickle_to_table(self, filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        data = data[1:]
        for idx, i in enumerate(data):
            data[idx][2] = i[2].replace('#', ' ').translate(self.non_bmp_map)
            data[idx][3] = '/'.join(i[3])
            data[idx][4] = '/'.join(i[4])
        self.raw_data = np.array(data)
  
    def hashtags_split(self, hashtags):        
        hashtags_split = []
        for i in hashtags:
            hashtags_split.append(i.split('/'))
        
        hashtags_list = []
        
        for i in hashtags_split:
            temp = []
            for j in i:
                if self.isHangul(j):
                    t_hashtags = j.translate(self.non_bmp_map)
                    temp.append(t_hashtags)
            hashtags_list.append(temp)
        self.hashtags_list = hashtags_list
        
        return hashtags_list

    
    def add_dictionary(self, *tokenized_list):
        origin_df = 1
        try:
            origin_df = pd.read_csv("C:\\mecab\\user-dic\\intake_dic.csv", encoding='utf-8', header=None)
        except:
            print('No default intake_dic')
        keyword_list = []
        for i in tokenized_list:
            if type(i) == list:
                for j in i:
                    j = j.split('_')
                    temp = [j[0],'' ,'' ,'' ,j[1],'*',j[2], j[3],'*','*','*','*','*']
                    keyword_list.append(temp)
            else:
                i = i.split('_')
                temp = [i[0],'','','',i[1],'*',i[2], i[3], '*','*','*','*','*']
                keyword_list.append(temp)


        keyword_df = pd.DataFrame(keyword_list)
        print(type(origin_df))
        if type(origin_df) != int:
            keyword_df = pd.concat((origin_df, keyword_df), ignore_index=True)
        else: 
            print('a')
            pass
        print(keyword_df.shape)

        keyword_df.to_csv("C:\\mecab\\user-dic\\intake_dic.csv", encoding='utf-8',index=None, header=False)    

        
    def ngram(self, parsed_list):
        ngram_list = []        
        adjustment = 0
        # 단어_tag의 리스트

        for idx in range(len(parsed_list)):
            idx2 = idx + adjustment

            if (idx2+self.ngram_size) > (len(parsed_list)):
                ngram_list.extend(parsed_list[idx2:])
                break
            n_filter = tuple(parsed_list[idx2: idx2 + self.ngram_size])
            key = ''.join([k.split('_')[0] for k in n_filter])
            if key in self.ngram_dic:
                ngram_list.append(self.ngram_dic[key])
                adjustment += (self.ngram_size - 1)
            else:
                ngram_list.append(n_filter[0])

        if self.ngram_size <= 1:
            return ngram_list
        else:
            self.ngram_size -= 1
            return self.ngram(ngram_list)       

    def set_with_order(self, sequence):
        seen = set()
        result = [x for x in sequence if not (x in seen or seen.add(x))]
        return result
    
    def morph_pos(self, text_list):
        
        morph_list = []
        
        for j in text_list:
            parsed = self.mecab.pos(j)
            temp = []
            for i in parsed:
                if self.isHangul(i[0]):
                    temp.append('{}_{}'.format(i[0], i[1]))
                else: pass#print('{} 한글이 아님.'.format(i[0]))

            self.ngram_size = 6
            morph_list.append(self.set_with_order(self.ngram(temp)))
        
        self.df['morph_list'] = morph_list
        return morph_list    
    
 
    def filter_words(self, parsed_list):
        # 1차원 리스트를 받음.
        changed_list = list(map(lambda x: self.syn_dic.get(x, x) , parsed_list))
        deleted_list = list(filter(lambda x: x not in self.del_list, changed_list))
        return deleted_list
    
    def pos_extractor(self, parsed, exception_list=['맛', '밥', '물', '몸', '없', '있', '싫', '달', '굳', '굿', '속']):

        noun_list = []
        adj_list = []
        verb_list = []
        nav_list = []
        for j in parsed:
            
            nav_temp = []
            n_temp = []
            adj_temp = []
            verb_temp = []
            
            for i in j:            
                i = i.split('_')
                if self.isHangul(i[0]):
                    if (len(i[0]) > 1) or (i[0] in exception_list):
                        if 'NN' in i[1]:
                            n_temp.append(i[0])
                            nav_temp.append(i[0])
                        elif 'VV'in i[1]:
                            adj_temp.append(i[0])
                            nav_temp.append(i[0])
                        elif 'VA' in i[1]:
                            verb_temp.append(i[0])
                            nav_temp.append(i[0])
                    else: pass
                        #print('{} 제외'.format(i[0]))
                else: pass#print('{} 한글이 아님.'.format(i[0]))

            nav_list.append(self.filter_words(nav_temp))
            noun_list.append(self.filter_words(n_temp))
            adj_list.append(self.filter_words(adj_temp))
            verb_list.append(self.filter_words(verb_temp))
            
            
        columns=['nav_list', 'noun_list', 'adj_list', 'verb_list']
        for i in  zip(columns, [nav_list, noun_list, adj_list, verb_list]):
            self.df[i[0]] = i[1]
            
        return nav_list, noun_list, adj_list, verb_list # tuple(map(lambda x: [j.split('_')[0] for j in x], [nav_list, noun_list, adj_list, verb_list]))

    
    def merge_list(self, tokenized_list):
        return [j for i in tokenized_list for j in i]

    
    def join_list(self, tokenized_list):
        joined_list = []
        for idx, i in enumerate(tokenized_list):
            joined_list.append(" ".join(i))
        return joined_list
 
    def split_list(self, untokenized_list):
        hashtag_splited = []
        for idx, i in enumerate(untokenized):
            hashtag_splited.append(i.split('/'))
            return hastag_splited
        
    '''    
    def join_underbar(self, morph_list):

        all_list = []
        post_list=[]
        for i in morph_list:
            for j in i:
                post_list.append(j[0]+'_'+j[1])
            all_list.append([(' , ').join(post_list)])
            post_list=[] 
        all_list=np.array(all_list)
        
        return all_list'''

    def word_substitute(self, dataset, sublist):
        dataset = copy.deepcopy(dataset)
        sub_book = dict()
        for i in sublist:
            for j in i['sub_words']:
                sub_book[j] = i['main']
        gc.collect()
        for n, i in enumerate(dataset):
            dataset[n] = [sub_book.get(item,item) for item in i]

        del sub_book
        gc.collect()

        return dataset
    
    def word_delete(self, dataset, del_list):
        dataset = copy.deepcopy(dataset)

        for n, line in enumerate(dataset):
             dataset[n] = [i for i in line if i not in del_list]

        return dataset

    
    def isHangul(self, text):
        encText = text
        hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
        return hanCount > 0
    
    def convert_list(self, *tokenized_list):
        input_length = len(tokenized_list)
        lists = [[] for i in range(input_length)]

        for idx, li in enumerate(tokenized_list):
            for j in li:
                lists[idx].append(['/'.join(j)])

        converted_array = np.array(lists[0])
        for idx in range(input_length):
            try:
                converted_array = np.concatenate((converted_array, lists[idx + 1]), axis=1)
            except Exception as e:
                print(e,'끝')

        return converted_array

    def make_df(self, start_array, converted_array, end_array, columns=['user_id', 'created_at', 'main_text', 'morph_list', 'nav_list', 'noun_list', 'adj_list', 'verb_list', 'hashtags', 'comments', 'likes', 'current_url']):         
        df = pd.DataFrame(np.hstack((start_array, converted_array, end_array)), index=None, columns=columns)
        return df
    
    # 키워드 리스트 중 하나라도 있는 경우
    def word_check_or(self, text, keywords):
        if any(word in text for word in keywords):
            return 1
        else: return 0

    # 키워드 리스트에 있는 단어가 모두 있는 경우
    def word_check_and(self, text, keywords):
        if all(word in text for word in keywords):
            return 1
        else:
            return 0


    def word_check(self, method, keywords, df, column_name = 'main_text',filter_TF=True):
        
        filter_TF = 1 if filter_TF == True else 0
        if method == 'and':
            df['flags'] = df[column_name].apply(lambda x: self.word_check_and(x, keywords))
            return df.loc[df['flags'] == filter_TF]

        elif method == 'or':
            df['flags'] = df[column_name].apply(lambda x: self.word_check_or(x, keywords))
            return df.loc[df['flags'] == filter_TF]
        
        else:
            print('Select method, and/or')

C:\Users\Jeongwoo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
itkinsta = Social_analysis()
itkshop = Social_analysis()
itkblog = Social_analysis()
lnsinsta = Social_analysis()
lnsshop = Social_analysis()
lnsblog = Social_analysis()
pckinsta = Social_analysis()
pckshop = Social_analysis()
pckblog = Social_analysis()
kgcinsta = Social_analysis()
kgcshop = Social_analysis()
kgcblog = Social_analysis()


In [5]:
itkinsta.insta_DB_to_table(DBname = 'intake', keyword = 'intake')
itkshop.shop_DB_to_table('intake', 'intake')
itkblog.blog_DB_to_table('intake', 'intake')
lnsinsta.insta_DB_to_table(DBname = 'intake', keyword = 'labnosh')
lnsshop.shop_DB_to_table('intake', 'labnosh')
lnsblog.blog_DB_to_table('intake', 'labnosh')
pckinsta.pickle_to_table('Data\pck_list.txt')
pckshop.shop_DB_to_table('intake', 'pck')
pckblog.blog_DB_to_table('intake', 'pck')
kgcinsta.pickle_to_table('Data\kgc_list.txt')
kgcshop.shop_DB_to_table('intake', 'kgc')
kgcblog.blog_DB_to_table('intake', 'kgc')


In [3]:
itkinsta.del_list.extend(['독립운동', '현정'])
lnsinsta.del_list.exten

In [6]:
print(kgcinsta.raw_data[0])

['chojungseok' '2017-07-13T23:27:15.000Z'
 'chojungseok 조정석  chojungseok  jojungsuk\n 정관장  정관장에브리타임  CF' ''
 '85_henny💗' ''
 'https://www.instagram.com/p/BWgVIRvA6zO/?hl=ko&tagged=%EC%A0%95%EA%B4%80%EC%9E%A5%EC%97%90%EB%B8%8C%EB%A6%AC%ED%83%80%EC%9E%84']


In [7]:
itkinsta.df = pd.DataFrame(itkinsta.raw_data, columns=['user_id', 'created_at', 'main_text', 'hashtags', 'comments', 'likes', 'current_url'])
itkshop.df = pd.DataFrame(itkshop.raw_data, columns=['mall', 'corpName', 'productCode', 'date', 'id', 'productScore', 'recommScore', 'main_text'])
itkblog.df = pd.DataFrame(itkblog.raw_data, columns=['keyword', 'created_at', 'post_name', 'main_text', 'current_url'])
lnsinsta.df = pd.DataFrame(lnsinsta.raw_data, columns=['user_id', 'created_at', 'main_text', 'hashtags', 'comments', 'likes', 'current_url'])
lnsshop.df = pd.DataFrame(lnsshop.raw_data, columns=['mall', 'corpName', 'productCode', 'date', 'id', 'productScore', 'recommScore', 'main_text'])
lnsblog.df = pd.DataFrame(lnsblog.raw_data, columns=['keyword', 'created_at', 'post_name', 'main_text', 'current_url'])
pckinsta.df = pd.DataFrame(pckinsta.raw_data, columns=['user_id', 'created_at', 'main_text', 'hashtags', 'comments', 'likes', 'current_url'])
pckshop.df = pd.DataFrame(pckshop.raw_data, columns=['mall', 'corpName', 'productCode', 'date', 'id', 'productScore', 'recommScore', 'main_text'])
pckblog.df = pd.DataFrame(pckblog.raw_data, columns=['keyword', 'created_at', 'post_name', 'main_text', 'current_url'])
kgcinsta.df = pd.DataFrame(kgcinsta.raw_data, columns=['user_id', 'created_at', 'main_text', 'hashtags', 'comments', 'likes', 'current_url'])
kgcshop.df = pd.DataFrame(kgcshop.raw_data, columns=['mall', 'corpName', 'productCode', 'date', 'id', 'productScore', 'recommScore', 'main_text'])
kgcblog.df = pd.DataFrame(kgcblog.raw_data, columns=['keyword', 'created_at', 'post_name', 'main_text', 'current_url'])

In [8]:
itkinsta.df = itkinsta.word_check('or', ['자동차', '흡기', '배기','도어락'], itkinsta.df, 'hashtags', False)
itkinsta.df = itkinsta.word_check('or', ['intakefoods', 'dameulstudio', '_.ddo2', '__scarlett.k', '0.8l_korea', 'jiseung86', 'untactmarket'], itkinsta.df, 'user_id', False)
itkinsta.df = itkinsta.word_check('or', ['현정', '독립운동', '배기','도어락'], itkinsta.df, 'main_text', False)


conver

temp = df_new_intake.loc[df_new['main_text'].str.contains('맛') & df_new['main_text'].str.contains('모닝죽') & df_new['main_text'].str.contains('있')]['']
temp.describe()

In [11]:
itkinsta.modified = itkinsta.df.as_matrix()
itkinsta.morph_list = itkinsta.morph_pos(itkinsta.modified[:, 2])

itkshop.modified = itkshop.df.as_matrix()
itkshop.morph_list = itkshop.morph_pos(itkshop.modified[:, 7])

itkblog.modified = itkblog.df.as_matrix()
itkblog.morph_list = itkblog.morph_pos(itkblog.modified[:, 3])

lnsinsta.modified = lnsinsta.df.as_matrix()
lnsinsta.morph_list = lnsinsta.morph_pos(lnsinsta.modified[:, 2])

lnsshop.modified = lnsshop.df.as_matrix()
lnsshop.morph_list = lnsshop.morph_pos(lnsshop.modified[:, 7])

lnsblog.modified = lnsblog.df.as_matrix()
lnsblog.morph_list = lnsblog.morph_pos(lnsblog.modified[:, 3])

pckinsta.modified = pckinsta.df.as_matrix()
pckinsta.morph_list = pckinsta.morph_pos(pckinsta.modified[:, 2])

pckshop.modified = pckshop.df.as_matrix()
pckshop.morph_list = pckshop.morph_pos(pckshop.modified[:, 7])

pckblog.modified = pckblog.df.as_matrix()
pckblog.morph_list = pckblog.morph_pos(pckblog.modified[:, 3])

kgcinsta.modified = kgcinsta.df.as_matrix()
kgcinsta.morph_list = kgcinsta.morph_pos(kgcinsta.modified[:, 2])

kgcshop.modified = kgcshop.df.as_matrix()
kgcshop.morph_list = kgcshop.morph_pos(kgcshop.modified[:, 7])

kgcblog.modified = kgcblog.df.as_matrix()
kgcblog.morph_list = kgcblog.morph_pos(kgcblog.modified[:, 3])


In [12]:
itkinsta.morph_list[0]

['식단_NNG',
 '일기_NNG',
 '아침_NNG',
 '인테이크_NNG',
 '소이_NNG',
 '밀크_NNP',
 '블랙_NNG',
 '점심_NNG',
 '아임_NNP',
 '웰_NNP',
 '굿_NNP',
 '밸런스_NNP',
 '라이스_NNP',
 '레드_NNP',
 '퀴노아_UNKNOWN',
 '갈릭_NNG',
 '닭_NNG',
 '가슴살_NNG',
 '훈제_NNG',
 '채소_NNG',
 '믹스_NNG',
 '브로콜리_NNP',
 '프랑크_NNP',
 '개_NNBC',
 '저녁_NNG',
 '곤약_NNG',
 '파스타_NNG',
 '꼬꼬_MAG',
 '빌_VV+ETM',
 '그릴_NNG',
 '사태_NNG',
 '수육_NNG',
 '등_NNB',
 '운동_NNG',
 '낸시_NNP',
 '홈_NNG',
 '트레이닝_NNG',
 '분_NNBC',
 '은_JX',
 '에_JKB',
 '과_JC',
 '를_JKO',
 '더_MAG',
 '해_VV+EC',
 '한번_MAG',
 '볶_VV',
 '아_EC',
 '주_VX',
 '었_EP',
 '음_ETN',
 '오뚜기_NNG',
 '프레스코_NNG',
 '스파게티_NNG',
 '소스_NNG',
 '와_JC',
 '토마토_NNG',
 '넣_VV',
 '어서_EC',
 '면_NNG',
 '쉐_NNG',
 '킷_NNP',
 '소시지_NNG',
 '빌_NNP']

In [13]:
itkinsta.nav_list, itkinsta.noun_list, itkinsta.adj_list, itkinsta.verb_list = itkinsta.pos_extractor(itkinsta.morph_list)
itkshop.nav_list, itkshop.noun_list, itkshop.adj_list, itkshop.verb_list = itkshop.pos_extractor(itkshop.morph_list)
itkblog.nav_list, itkblog.noun_list, itkblog.adj_list, itkblog.verb_list = itkblog.pos_extractor(itkblog.morph_list)

lnsinsta.nav_list, lnsinsta.noun_list, lnsinsta.adj_list, lnsinsta.verb_list = lnsinsta.pos_extractor(lnsinsta.morph_list)
lnsshop.nav_list, lnsshop.noun_list, lnsshop.adj_list, lnsshop.verb_list = lnsshop.pos_extractor(lnsshop.morph_list)
lnsblog.nav_list, lnsblog.noun_list, lnsinsta.adj_list, lnsblog.verb_list = lnsblog.pos_extractor(lnsblog.morph_list)

pckinsta.nav_list, pckinsta.noun_list, pckinsta.adj_list, pckinsta.verb_list = pckinsta.pos_extractor(pckinsta.morph_list)
pckshop.nav_list, pckshop.noun_list, pckshop.adj_list, pckshop.verb_list = pckshop.pos_extractor(pckshop.morph_list)
pckblog.nav_list, pckblog.noun_list, pckblog.adj_list, pckblog.verb_list = pckblog.pos_extractor(pckblog.morph_list)

kgcinsta.nav_list, kgcinsta.noun_list, kgcinsta.adj_list, kgcinsta.verb_list = kgcinsta.pos_extractor(kgcinsta.morph_list)
kgcshop.nav_list, kgcshop.noun_list, kgcshop.adj_list, kgcshop.verb_list = kgcshop.pos_extractor(kgcshop.morph_list)
kgcblog.nav_list, kgcblog.noun_list, kgcblog.adj_list, kgcblog.verb_list = kgcblog.pos_extractor(kgcblog.morph_list)

In [16]:
itkinsta.nav_merged = itkinsta.merge_list(itkinsta.nav_list)
itkshop.nav_merged = itkshop.merge_list(itkshop.nav_list)
itkblog.nav_merged = itkblog.merge_list(itkblog.nav_list)

lnsinsta.nav_merged = lnsinsta.merge_list(lnsinsta.nav_list)
lnsshop.nav_merged = lnsshop.merge_list(lnsshop.nav_list)
lnsblog.nav_merged = lnsblog.merge_list(lnsblog.nav_list)

pckinsta.nav_merged = pckinsta.merge_list(pckinsta.nav_list)
pckshop.nav_merged = pckshop.merge_list(pckshop.nav_list)
pckblog.nav_merged = pckblog.merge_list(pckblog.nav_list)

kgcinsta.nav_merged = kgcinsta.merge_list(kgcinsta.nav_list)
kgcshop.nav_merged = kgcshop.merge_list(kgcshop.nav_list)
kgcblog.nav_merged = kgcblog.merge_list(kgcblog.nav_list)


In [17]:
itkinsta.temp_df = itkinsta.word_check('or',['독립운동'],itkinsta.df)

In [18]:
itkinsta.temp_df

,user_id,created_at,main_text,hashtags,comments,likes,current_url,flags,morph_list,nav_list,noun_list,adj_list,verb_list


In [26]:
pckinsta.temp_df = lnsinsta.word_check('or',['우바'],lnsinsta.df)

In [29]:
lnsinsta.temp_df

,user_id,created_at,main_text,hashtags,comments,likes,current_url,morph_list,nav_list,noun_list,adj_list,verb_list,flags
1,hjw2151,2017-01-10 13:09:17,hjw2151교무실 내책상~😘 방학이라서 좋구나~~^^ 방학이지만 열심히 생활기록부 작성중!\n매일 아침식사는 가볍지만 완벽한 식사\n랩노쉬의 The Future Food.\n여자의 마음을 사로잡는 랩노쉬💕\n하나만 마셔도 배부름 .\n.\n#랩노쉬 #LABNOSH #The_future_food #미래형식사 #올리브영 #방학 #겨울방학 #맛있다 #맛있다그램 #수국 #수국꽃다발 #대일외고 #교무실 #토마토 #그래놀라요거트 #그레인 #그린씨리얼 #우바밀크 #자색고구마 #쇼콜라 #쉐킷쉐킷 #shake_it #간편식 #다이어트 #건강식 #먹스타그램 #아침식사대용 #대용식 #soldout #완판,랩노쉬/LABNOSH/The_future_food/미래형식사/올리브영/방학/겨울방학/맛있다/맛있다그램/수국/수국꽃다발/대일외고/교무실/토마토/그래놀라요거트/그레인/그린씨리얼/우바밀크/자색고구마/쇼콜라/쉐킷쉐킷/shake_it/간편식/다이어트/건강식/먹스타그램/아침식사대용/대용식/soldout/완판,ssun_unni요거 배부르면서 든든하더라구여👍/leesimok잘보고가요~✌/iam_what.i.am쌤 저 올리브영 알바하는데 저거 진짜 잘나가요😂😂/hjw2151대박....진짜? 올리브영에서 알바해? 기특하다 우리연경이 화이팅~~💓/apilates9사진들 잘보고 갑니닷~~ 또 뵈어요~👏,58,https://www.instagram.com/p/BPFcJOpAmwF/?hl=ko&tagged=%EB%9E%A9%EB%85%B8%EC%89%AC,"[교무실_NNG, 내_NNB, 책상_NNG, 방학_NNG, 이_VCP, 라서_EC, 좋_VA, 구나_EF, 지만_EC, 열심히_MAG, 생활_NNG, 기록부_NNG, 작성_NNG, 중_NNB, 매일_MAG, 아침_NNG, 식사_NNG, 는_JX, 가볍_VA, 완벽_NNG, 한_XSA+ETM, 랩노쉬_NNG, 의_JKG, 여자_NNG, 마음_NNG, 을_JKO, 사로잡_VV, 는_ETM, 하나_NR, 만_JX, 마셔_VV+EC, 도_JX, 배부름_VA+ETN, 미래형_NNG, 올리브_NNG, 영_NNG, 겨울_NNG, 맛있_VA, 다_EC, 그램_NNG, 수국_NNG, 꽃다발_NNG, 대일_NNP, 외고_NNP, 토마토_NNG, 그래놀라_NNG, 요거트_NNG, 그레인_NNG, 그린_VV+ETM, 씨리얼_NNG, 우바_NNP, 밀크_NNP, 자색_NNG, 고구마_NNG, 쇼콜라_NNP, 쉐_NNG, 킷_NNP, 간편식_NNG, 다이어트_NNG, 건강식_NNG, 먹스타그램_NNG, 식사대용_NNG, 대용식_NNG, 완_NNP, 판_NNG]","[교무실, 책상, 방학, 생활, 기록부, 작성, 아침, 식사, 가볍, 완벽, 랩노쉬, 여자, 마음, 사로잡, 마셔, 배부름, 미래형, 올리브, 겨울, 맛있, 그램, 수국, 꽃다발, 대일, 외고, 토마토, 그래놀라, 요거트, 그레인, 그린, 씨리얼, 우바, 밀크, 자색, 고구마, 쇼콜라, 간편식, 다이어트, 건강식, 먹스타그램, 식사대용, 대용식]","[교무실, 책상, 방학, 생활, 기록부, 작성, 아침, 식사, 완벽, 랩노쉬, 여자, 마음, 미래형, 올리브, 겨울, 그램, 수국, 꽃다발, 대일, 외고, 토마토, 그래놀라, 요거트, 그레인, 씨리얼, 우바, 밀크, 자색, 고구마, 쇼콜라, 간편식, 다이어트, 건강식, 먹스타그램, 식사대용, 대용식]","[사로잡, 마셔, 그린]","[가볍, 배부름, 맛있]",1
8,kimsweet96,2017-01-10 05:32:05,"kimsweet96출근 3주차, 살기 위해 랩노쉬를 구입하기 시작했다\n.\n.\n#랩노쉬 #미래형식사 #우바밀크티 #든든함 #편함 #3900원?! #맛있음 #파랑파랑 #먹스타그램 #맛있는거 #먹고싶다 #칼퇴희망",랩노쉬/미래형식사/우바밀크티/든든함/편함/3900원/맛있음/파랑파랑/먹스타그램/맛있는거/먹고싶다/칼퇴희망/ryuha96,lalalse__수정이바쁘게살구잇져?/hannayaaat친하게 지내도 될까요^^?? 인친해요~🎉/kimsweet96@ryuha96 학생들이 너무 많은 학원에서 근무하고이쩡....😢/lalalse__잘챙겨먹어야돼ㅠㅠㅠㅠ,15,https://www.instagram.com/p/BPEn0gzDtxT/?hl=ko&tagged=%EB%9E%A9%EB%85%B8%EC%89%AC,"[출근_NNG, 주_NNBC, 차_XSN, 살_VV, 기_ETN, 위해_VV+EC, 랩노쉬_NNG, 를_JKO, 구입_NNG, 하_XSV, 시작_NNG, 했_XSV+EP, 다_EF, 미래형_NNG, 식사_NNG, 우바_NNP, 밀크티_NNP, 든든_XR, 함_XSA+ETN, 편함_VA+ETN, 원_NNBC, 맛있_VA, 음_ETN, 파랑_NNG, 먹스타그램_NNG, 는_ETM, 거_NNB, 먹_VV, 고_EC, 싶_VX, 다_EC, 칼_NNG, 퇴_NNG, 희망_NNG]","[출근, 위해, 랩노쉬, 구입, 시작, 미래형, 식사, 우바, 밀크티, 편함, 맛있, 파랑, 먹스타그램, 희망]","[출근, 랩노쉬, 구입, 시작, 미래형, 식사, 우바, 밀크티, 파랑, 먹스타그램, 희망]",[위해],"[편함, 맛있]",1
16,ssun_unni,2017-01-09 12:49:40,ssun_unni#오늘저녁 은 #랩노쉬 #우바밀크티\n맛있게 먹을수있는 #미래형식사 랩노쉬\n은근 든든하면서 배부르당 ㅎㅎ\n하지만 야식이 땡기는밤...\n너므 일찍먹었나 ㅠㅠ꼬륵..😭\n.\n.\n.\n.\n.\n#다이어트 #랩노쉬우바밀크티 #맞팔 #인친,오늘저녁/랩노쉬/우바밀크티/미래형식사/다이어트/랩노쉬우바밀크티/맞팔/인친/jayz_effect,jaysanglee밥을!!!! 먹으시오!!!!!/ssun_unni@jayz_effect 밥이예여 이거 ㅋㅋㅋㅋ미래형식사임ㅋㅋㅋㅋㅋㅋㅋㅋ/jaysanglee아직 미래아니자나여... 밥먹어여../seoulo99좋은하루여 ^_^😸😃/mini._.store잘봤습니다! 팔로워 신청 해두될까요?✌/the_zzugumi_official구경잘하고가요😄✌,119,https://www.instagram.com/p/BPC1GmQhuRO/?hl=ko&tagged=%EB%9E%A9%EB%85%B8%EC%89%AC,"[오늘_NNG, 저녁_NNG, 은_JX, 랩노쉬_NNG, 우바_NNP, 밀크티_NNP, 맛있_VA, 게_EC, 먹_VV, 을_ETM, 수_NNB, 있_VV, 는_ETM, 미래형_NNG, 식사_NNG, 은근_NNG, 든든_XR, 하_XSA, 면서_EC, 배부르_VA, 당_EF, ㅎㅎ_IC, 하지만_MAJ, 야식_NNG, 이_JKS, 땡기_VV, 밤_NNG, 너_NP, 므_UNKNOWN, 일찍_MAG, 었_EP, 나_EF, ㅠㅠ꼬륵_UNKNOWN, 다이어트_NNG, 맞팔_NNG, 인친_NNG]","[오늘, 저녁, 랩노쉬, 우바, 밀크티, 맛있, 있, 미래형, 식사, 은근, 배부르, 야식, 땡기, 다이어트, 맞팔, 인친]","[오늘, 저녁, 랩노쉬, 우바, 밀크티, 미래형, 식사, 은근, 야식, 다이어트, 맞팔, 인친]","[있, 땡기]","[맛있, 배부르]",1
19,ppo.d,2017-01-09 09:00:27,ppo.d2017. 01. 09\n저녁시단.\n.\n▫랩노쉬 우바 밀크티 (@labnosh_official).\n.\n나 이거 먹어야되는데\n오늘 왜자꾸 바쁨..\n상담이 너무 길어져서 화장실 갈 시간도 밥 먹을 시간도 없음😢.\n.\n.\n퇴근하고 무거야지 엉엉\n.\n.\n#다이어트#다이어터#다이어트그램#다이어트계정#다이어트식단#식단조절#식이조절#다이어트기록#다이어트일기#식단일기#식단기록#식단관리#식단공유#랩노쉬#미래지향식사#미래형식사,labnosh_official/다이어트/다이어터/다이어트그램/다이어트계정/다이어트식단/식단조절/식이조절/다이어트기록/다이어트일기/식단일기/식단기록/식단관리/식단공유/랩노쉬/미래지향식사/미래형식사,hsssssy_02왜우

In [19]:
itk_merged = itkinsta.nav_merged + itkshop.nav_merged + itkblog.nav_merged
lns_merged = lnsinsta.nav_merged + lnsshop.nav_merged + lnsblog.nav_merged
pck_merged = pckinsta.nav_merged + pckshop.nav_merged + pckblog.nav_merged
kgc_merged = kgcinsta.nav_merged + kgcshop.nav_merged + kgcblog.nav_merged

In [20]:
itk_merged

['식단',
 '일기',
 '아침',
 '인테이크',
 '소이',
 '밀크',
 '블랙',
 '점심',
 '아임',
 '굿',
 '밸런스',
 '라이스',
 '레드',
 '갈릭',
 '가슴살',
 '훈제',
 '채소',
 '믹스',
 '브로콜리',
 '프랑크',
 '저녁',
 '곤약',
 '파스타',
 '그릴',
 '사태',
 '수육',
 '운동',
 '낸시',
 '트레이닝',
 '오뚜기',
 '프레스코',
 '스파게티',
 '소스',
 '토마토',
 '소시지',
 '요즘',
 '유산균',
 '인테이크',
 '판매',
 '주문',
 '밀스',
 '괜찮',
 '보통',
 '알약',
 '개별',
 '포장',
 '가루',
 '물',
 '흔들',
 '마시',
 '타입',
 '마음',
 '처음',
 '따라서',
 '용기',
 '과정',
 '귀찮',
 '분유',
 '맛',
 '마지막',
 '단게',
 '설탕',
 '알갱이',
 '바닥',
 '마셔야',
 '바뀌',
 '덕분',
 '모르',
 '화장실',
 '달',
 '장복',
 '걸로',
 '휴먼',
 '바이오',
 '예쁘',
 '효과',
 '인테이크',
 '올해',
 '건강',
 '이러',
 '처먹',
 '식단',
 '일기',
 '마음잡',
 '시작',
 '아침',
 '인테이크',
 '소이밀크퓨어100',
 '점심',
 '아임',
 '라이트',
 '카레',
 '큐브',
 '간식',
 '반숙',
 '계란',
 '저녁',
 '스테이크',
 '매콤',
 '단호박',
 '토마토',
 '브로콜리',
 '풀무원',
 '낫토',
 '파리',
 '바게트',
 '미니',
 '치즈',
 '케익',
 '운동',
 '기구',
 '필라테스',
 '재미있',
 '굿',
 '소이',
 '밀크',
 '토미',
 '오늘',
 '식단',
 '저녁',
 '인테이크',
 '모닝죽단팥',
 '냉동야채',
 '가슴살',
 '소세지',
 '일기',
 '모닝죽',
 '뷰티',
 '실험실',
 '다이어트',
 '건강',
 '도와',
 '인테이크',
 '가지',
 

In [28]:
pd.options.display.max_colwidth = 2000

In [23]:
tfidf = SB_Tfidf([itk_merged, lns_merged, pck_merged, kgc_merged])
tfidf.get_tfidf()
tfidf_of_all = tfidf.tfidf_hangul

for i in tfidf_of_all:
    pprint(i[:10])
    print()

[('인테이크', 0.7686450126935154),
 ('밀스', 0.4225245299265768),
 ('모닝죽', 0.20284349684017755),
 ('식사대용', 0.1694863047208061),
 ('고구마죽', 0.12170543525059005),
 ('밀스라이트', 0.11366828386611712),
 ('아침대용', 0.09673585714617706),
 ('미래식사', 0.09414948764668286),
 ('슈퍼바', 0.0849641717787138),
 ('파워젤부스트', 0.08343328580071896)]

[('랩노쉬', 0.7996747379834489),
 ('미래형', 0.44431849149108354),
 ('푸드쉐이크', 0.16086558252457986),
 ('미식당', 0.1509326173385742),
 ('식사대용', 0.1392618523962123),
 ('우바', 0.12282959388548492),
 ('쇼콜라', 0.11140948191696987),
 ('스타터키트', 0.08430907035926777),
 ('그래놀라', 0.0793340083325715),
 ('랩노쉬플랫', 0.0744070547098896)]

[('피코', 0.7584170716936725),
 ('코크', 0.5981067236051538),
 ('초마', 0.1436172405541844),
 ('포레스티', 0.06984126081744584),
 ('호떡', 0.06344734257359516),
 ('노브랜드', 0.04878754503127364),
 ('서주현', 0.039347189192927234),
 ('하노이', 0.03737982973328087),
 ('라자냐', 0.03147775135434178),
 ('녹두', 0.030494071624518605)]

[('에브리타임', 0.8736434164169138),
 ('정관장', 0.45374306562829664),
 

## csv에 안쓰고 이런식으로도 추가 가능. 하지만 csv에 자동저장되진 않는다.

단어_태그_T/F(받침여부)_원단어의발음

## m이 False인 경우에는 진짜 맛있다는 것.
## -> '없' 이라는 음절이 '있'이라는 음절 이후 4음절 내에 없다는 뜻이다.
## m이 True인 경우에는 애매한 상황이다. '없' 이라는 음절이 4음절 내에 존재한다는 것이다.

In [20]:
import re
m = re.search('맛.{0,5}있.{0,5}[없않]', '안녕하세요. 맛이 있진 않다.. 다만 아쉬운 것은 포장지가 없다.')
print(m)
print('맛있다는 뜻' if m == None else '맛 없다는 뜻\n원문: ' + m.group())

m = re.search('맛.{0,5}있.{0,5}[없않]', '안녕하세요. 맛이 있다. 다만 아쉬운 것은 포장지가 없다.')
print(m)
print('맛있다는 뜻' if m == None else '맛 없다는 뜻\n원문: ' + m.group())


<_sre.SRE_Match object; span=(7, 14), match='맛이 있진 않'>
맛 없다는 뜻
원문: 맛이 있진 않
None
맛있다는 뜻
